In [1]:
import os
import time
FOLDERNAME = "en-vi-sent2vec-OpenNMT-OPUS-EM-0.6-" + str(time.strftime("%Y%m%d-%H%M"))
!mkdir $FOLDERNAME

os.chdir(FOLDERNAME)
!pwd

# import os
# path = '/content/drive/Shared drives/chinh-share/nmt-v3/OpenNMT-TED-EM-bert-ratio-8-2-2-20210128-0637'
# os.chdir(path)
# !pwd

/home/chinh/clc_data/Script/en-vi-sent2vec-OpenNMT-OPUS-EM-0.6-20210302-2324


In [2]:
!wget https://github.com/hoangtrungchinh/clc_data/raw/master/opus-100-corpus-en-vi-EM.zip
!mkdir data_bin
!unzip 'opus-100-corpus-en-vi-EM.zip'

--2021-03-02 23:24:22--  https://github.com/hoangtrungchinh/clc_data/raw/master/opus-100-corpus-en-vi-EM.zip
Resolving github.com (github.com)... 52.74.223.119
Connecting to github.com (github.com)|52.74.223.119|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/opus-100-corpus-en-vi-EM.zip [following]
--2021-03-02 23:24:23--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/opus-100-corpus-en-vi-EM.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93465891 (89M) [application/zip]
Saving to: ‘opus-100-corpus-en-vi-EM.zip’

opus-100-corpus-en- 100%[===================>]  89,14M  6,45MB/s    in 16s     

2021-03-02 23:24:45 (5,42 MB/s) - ‘opus-100-c

In [3]:
!mkdir -p output
!onmt_preprocess -train_src 'en_train_EM_0.6' -train_tgt 'vi_train' -valid_src 'en_valid' -valid_tgt 'vi_valid' -save_data 'output/en-vi' 

[2021-03-02 23:24:48,601 INFO] Extracting features...
[2021-03-02 23:24:48,601 INFO]  * number of source features: 0.
[2021-03-02 23:24:48,601 INFO]  * number of target features: 0.
[2021-03-02 23:24:48,601 INFO] Building `Fields` object...
[2021-03-02 23:24:48,601 INFO] Building & saving training data...
multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/home/chinh/anaconda3/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/home/chinh/anaconda3/lib/python3.8/site-packages/onmt/bin/preprocess.py", line 54, in process_one_shard
    assert len(src_shard) == len(tgt_shard)
AssertionError
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/chinh/anaconda3/bin/onmt_preprocess", line 8, in <module>
    sys.exit(main())
  File "/home/chinh/anaconda3/lib/python3.8/site-packages/onmt/bin/preprocess.py", line 318, in main
    prep

In [ ]:
!mkdir -p model
!onmt_train -data 'output/en-vi' \\
-save_model 'model/en-vi' \\
-layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8 \\
-encoder_type transformer -decoder_type transformer -position_encoding \\
-train_steps 30000  -max_generator_batches 2 -dropout 0.1 -batch_size 4096 \\
-batch_type tokens -normalization tokens  -accum_count 2 -optim adam -adam_beta2 0.998 \\
-decay_method noam -warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 \\
-param_init_glorot -label_smoothing 0.1 -valid_steps 1000 -save_checkpoint_steps 1000 \\
-report_every 1000 -world_size 1 -gpu_ranks 0

In [ ]:
!ls -al model model/

In [ ]:
!onmt_translate -model model/en-vi_step_30000.pt -src en_test -tgt vi_test -output predict.txt

In [ ]:
!tail vi_test

In [ ]:
!git clone https://github.com/OpenNMT/OpenNMT-py.git

In [ ]:
!ls -al

In [ ]:
!perl OpenNMT-py/tools/multi-bleu.perl vi_test < predict.txt